### Load tensorflow

In [0]:
import tensorflow as tf

In [15]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns=['date','symbol'])

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = data[:1000]

### Convert Float64 to Float32

In [0]:
for col in data.columns:
  data[col]=data[col].astype('float32')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float32
close     1000 non-null float32
low       1000 non-null float32
high      1000 non-null float32
volume    1000 non-null float32
dtypes: float32(5)
memory usage: 19.6 KB


### Divide the data into train and test sets

In [0]:
X=data.drop(columns='close')
Y=data['close']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.25, random_state=123)

In [108]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(750, 4)
(750,)
(250, 4)
(250,)


### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(X_train)

In [0]:
import numpy as np

In [111]:
train_x[2]

array([1.1791080e-05, 1.1639602e-05, 1.1888028e-05, 1.0000000e+00],
      dtype=float32)

In [112]:
y_train.shape

(750,)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [118]:
for i in range(100):
    
    w, b = train(train_x, np.array(y_train), w, b)
    print('Current Loss on iteration', i, loss(np.array(y_train), prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 7813.3086
Current Loss on iteration 1 7507.9565
Current Loss on iteration 2 7226.5474
Current Loss on iteration 3 6967.197
Current Loss on iteration 4 6728.1816
Current Loss on iteration 5 6507.905
Current Loss on iteration 6 6304.898
Current Loss on iteration 7 6117.8057
Current Loss on iteration 8 5945.3804
Current Loss on iteration 9 5786.477
Current Loss on iteration 10 5640.0293
Current Loss on iteration 11 5505.063
Current Loss on iteration 12 5380.6797
Current Loss on iteration 13 5266.047
Current Loss on iteration 14 5160.3984
Current Loss on iteration 15 5063.0376
Current Loss on iteration 16 4973.3066
Current Loss on iteration 17 4890.6133
Current Loss on iteration 18 4814.4014
Current Loss on iteration 19 4744.163
Current Loss on iteration 20 4679.434
Current Loss on iteration 21 4619.779
Current Loss on iteration 22 4564.8
Current Loss on iteration 23 4514.1333
Current Loss on iteration 24 4467.437
Current Loss on iteration 25 4424.402
Current Lo

### Get the shapes and values of W and b

In [120]:
#Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5769507e-03]
 [2.5534146e-03]
 [2.5974149e-03]
 [3.3310249e+01]]
Bias:
 [33.31027]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [122]:
model.fit(train_x, np.array(y_train), epochs=100)

Epoch 1/100
750/750 [==============================] - 0s 431us/sample - loss: 6027.3004
Epoch 2/100
750/750 [==============================] - 0s 39us/sample - loss: 4216.9186
Epoch 3/100
750/750 [==============================] - 0s 34us/sample - loss: 3963.4632
Epoch 4/100
750/750 [==============================] - 0s 30us/sample - loss: 3925.3932
Epoch 5/100
750/750 [==============================] - 0s 31us/sample - loss: 3924.2885
Epoch 6/100
750/750 [==============================] - 0s 34us/sample - loss: 3922.3123
Epoch 7/100
750/750 [==============================] - 0s 28us/sample - loss: 3923.6762
Epoch 8/100
750/750 [==============================] - 0s 33us/sample - loss: 3923.1044
Epoch 9/100
750/750 [==============================] - 0s 39us/sample - loss: 3923.2019
Epoch 10/100
750/750 [==============================] - 0s 41us/sample - loss: 3922.5754
Epoch 11/100
750/750 [==============================] - 0s 33us/sample - loss: 3922.6156
Epoch 12/100
750/750 [=======

In [123]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Iris_data = pd.read_csv('./Iris.csv')

In [126]:
Iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [132]:
Iris_data['Species'].value_counts()

Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: Species, dtype: int64

### Splitting the data into feature set and target set

In [0]:
feature_set=Iris_data.drop(columns='Species')
target_set=Iris_data['Species']

In [170]:
feature_set=feature_set.drop(columns='Id')
feature_set.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
target_set= label_encoder.fit_transform(target_set)

In [173]:
target_set

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [0]:
target_set_1 = tf.keras.utils.to_categorical(target_set, num_classes=3)

In [175]:
target_set_1[2]

array([1., 0., 0.], dtype=float32)

### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(feature_set, target_set_1, test_size=.25, random_state=123)

In [177]:
X_train.shape

(112, 4)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))


#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [185]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=10)

Train on 112 samples, validate on 38 samples
Epoch 1/100
112/112 [==============================] - 1s 5ms/sample - loss: 1.0769 - acc: 0.3839 - val_loss: 1.1055 - val_acc: 0.2105
Epoch 2/100
112/112 [==============================] - 0s 351us/sample - loss: 1.0766 - acc: 0.3750 - val_loss: 1.1084 - val_acc: 0.2105
Epoch 3/100
112/112 [==============================] - 0s 421us/sample - loss: 1.0771 - acc: 0.3750 - val_loss: 1.1103 - val_acc: 0.2105
Epoch 4/100
112/112 [==============================] - 0s 302us/sample - loss: 1.0747 - acc: 0.3750 - val_loss: 1.1122 - val_acc: 0.2105
Epoch 5/100
112/112 [==============================] - 0s 264us/sample - loss: 1.0718 - acc: 0.3750 - val_loss: 1.1062 - val_acc: 0.2105
Epoch 6/100
112/112 [==============================] - 0s 338us/sample - loss: 1.0745 - acc: 0.3661 - val_loss: 1.1037 - val_acc: 0.2105
Epoch 7/100
112/112 [==============================] - 0s 325us/sample - loss: 1.0732 - acc: 0.3750 - val_loss: 1.0974 - val_acc: 0.210

### Report Accuracy of the predicted values

In [186]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 4)                 16        
_________________________________________________________________
dense_6 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


In [187]:
pred = model.predict(X_test)
pred

array([[0.2277635 , 0.42395595, 0.34828058],
       [0.20296386, 0.3705678 , 0.42646837],
       [0.19535111, 0.38157818, 0.42307067],
       [0.29253706, 0.37705976, 0.3304032 ],
       [0.43154126, 0.34032154, 0.22813724],
       [0.23339635, 0.37476864, 0.39183503],
       [0.23834832, 0.4421823 , 0.31946933],
       [0.43551886, 0.33354124, 0.23093992],
       [0.41817328, 0.35109782, 0.23072885],
       [0.27322915, 0.4202402 , 0.30653068],
       [0.19849615, 0.35830566, 0.44319817],
       [0.45346504, 0.29000852, 0.25652647],
       [0.2749711 , 0.40385255, 0.32117632],
       [0.22145097, 0.37862703, 0.399922  ],
       [0.1910987 , 0.3669871 , 0.44191414],
       [0.20035022, 0.37034407, 0.42930573],
       [0.44222945, 0.31704682, 0.24072371],
       [0.43054116, 0.34260783, 0.22685103],
       [0.25732702, 0.39233243, 0.35034055],
       [0.41590974, 0.3471228 , 0.23696747],
       [0.44010028, 0.323172  , 0.2367277 ],
       [0.23726232, 0.4012213 , 0.36151642],
       [0.

In [202]:
model.evaluate(X_test,y_test)

38/38 [==============================] - 0s 3ms/sample - loss: 0.8642 - acc: 0.9474


[0.8642018500127291, 0.94736844]

In [0]:
#Accuracy on test is 94%